In [2]:
# !pip install fast_cdindex

ERROR: Could not find a version that satisfies the requirement fast_cdindex (from versions: none)
ERROR: No matching distribution found for fast_cdindex


## CD index

In [29]:
# pip install git+https://github.com/dspinellis/fast-cdindex.git

In [1]:
from fast_cdindex import cdindex
import datetime
import pandas as pd
# from datetime import datetime
from tqdm import tqdm

## Import data

### Patents

In [156]:
# Path to the .csv.gz file
file_path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_patent.tsv"

# Load the TSV file into a DataFrame
patent_df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

# Display the first few rows of the DataFrame
print(patent_df.shape)
patent_df.head()

C:\Users\edgar\AppData\Local\Temp\ipykernel_28968\3886042248.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  patent_df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')


(8980130, 8)


,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


#### Traitement données

In [157]:
patent_df["patent_date"] = pd.to_datetime(patent_df["patent_date"])
patent_df = patent_df[(patent_df["patent_type"]== "utility") & (patent_df["patent_date"].dt.year <= 2010)]

patent_df = patent_df[patent_df["patent_id"].notna()]

# Step 2: Ensure "patent_id" is treated as string for numeric filtering
patent_df["patent_id"] = patent_df["patent_id"].astype(str)
# Step 3: Keep rows where "patent_id" contains only numeric values (without NaN)
patent_df = patent_df[patent_df["patent_id"].str.isnumeric()]

# Step 4: Optionally, convert "patent_id" back to numeric type
patent_df["patent_id"] = pd.to_numeric(patent_df["patent_id"])
# patent_df[patent_df["patent_id"]=="RE31700"]

print(patent_df.shape)

(3912353, 8)


In [158]:
# mettre en string
patent_df["patent_id"] = patent_df["patent_id"].astype(str)
valid_patent_ids = set(patent_df["patent_id"])

In [5]:
patent_df.shape

(3912353, 8)

In [96]:
# Convert 'patent_date' to datetime format (efficiently)
patent_df['patent_date'] = pd.to_datetime(patent_df['patent_date']).dt.tz_localize('UTC')

# Use a vectorized approach to create the dictionary list
pyvertices = patent_df[['patent_id', 'patent_date']].rename(columns={'patent_id': 'name', 'patent_date': 'time'}).to_dict(orient="records")

# Print result
pyvertices[2_000_000:2_000_005]

[{'name': '12016711', 'time': Timestamp('2024-06-25 00:00:00+0000', tz='UTC')},
 {'name': '12016712', 'time': Timestamp('2024-06-25 00:00:00+0000', tz='UTC')},
 {'name': '12016713', 'time': Timestamp('2024-06-25 00:00:00+0000', tz='UTC')},
 {'name': '12016714', 'time': Timestamp('2024-06-25 00:00:00+0000', tz='UTC')},
 {'name': '12016715', 'time': Timestamp('2024-06-25 00:00:00+0000', tz='UTC')}]

In [101]:
pyvertices[1]

{'name': '3930272', 'time': Timestamp('1976-01-06 00:00:00+0000', tz='UTC')}

In [22]:
graph.in_degree('5941792')

13

In [26]:
graph.timestamp('5941794')

935452800

In [25]:
pd.to_datetime(graph.timestamp('5941794'), unit='s')

935452800

### Cd-Index Fast

In [6]:
# Graph
# create graph

graph = cdindex.Graph()

In [7]:
# add vertices
for vertex in tqdm(pyvertices, mininterval=1.0):
    graph.add_vertex(vertex["name"], int(datetime.datetime.timestamp(vertex["time"])))

100%|██████████| 8123317/8123317 [00:23<00:00, 345619.82it/s]


In [8]:
del pyvertices
valid_patent_ids = set(patent_df["patent_id"])
del(patent_df)

#### Granted

In [9]:
chunksize = 10_000_000  # Read 10 million rows at a time
total_rows = 0  # Counter to track progress
grant_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_patent_citation.tsv"

for chunk in pd.read_csv(grant_file, sep='\t', usecols=["patent_id", "citation_patent_id"], chunksize=chunksize, on_bad_lines='skip'):
    print(f"Processing rows {total_rows} to {total_rows + len(chunk)}")
    
    # Convert to string if needed
    chunk["patent_id"] = chunk["patent_id"].astype(str)
    chunk["citation_patent_id"] = chunk["citation_patent_id"].astype(str)

    filtered_df = chunk[chunk["patent_id"].isin(valid_patent_ids) & chunk["citation_patent_id"].isin(valid_patent_ids)]
    
    pyedges = filtered_df.rename(columns={'patent_id': 'source', 'citation_patent_id': 'target'}) \
            .astype(str) \
            .to_dict(orient='records')
    
    # add edges
    for edge in tqdm(pyedges, mininterval=1.0):
        graph.add_edge(edge["source"], edge["target"])


    total_rows += len(chunk)
    
    # Optional: Save processed chunk to a new file
    # chunk.to_csv("output_file.tsv", mode='a', sep='\t', index=False, header=(total_rows == 0))  # Append mode

print(f"Finished processing {total_rows} rows.")
del(chunk)
del filtered_df
del pyedges
# 22 min

Processing rows 0 to 10000000


100%|██████████| 9281578/9281578 [00:17<00:00, 539028.61it/s]


Processing rows 10000000 to 20000000


100%|██████████| 9301220/9301220 [00:18<00:00, 509314.46it/s]


Processing rows 20000000 to 30000000


...

Processing rows 130000000 to 140000000


0it [00:00, ?it/s]


Processing rows 140000000 to 144025878


0it [00:00, ?it/s]


Finished processing 144025878 rows.


In [ ]:
# applic_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_application_citation.tsv"
# for_df=pd.read_csv(applic_file, sep='\t',nrows=1000)
# for_df

,patent_id,citation_sequence,citation_document_number,citation_date,record_name,wipo_kind,citation_category
0,8102431,0,20070236734,2007-10-01,Okamoto,A1,cited by examiner
1,8117225,0,20040158607,2004-08-01,Coppinger et al.,A1,cited by other
2,8717974,0,20070229270,2007-10-01,Rofougaran,A1,cited by applicant
3,8182067,0,20040036751,2004-02-01,Giere et al.,A1,cited by examiner
4,10629046,104,20120212582,2012-08-01,Deutsch,A1,cited by applicant
...,...,...,...,...,...,...,...
995,11908515,29,20130286763,2013-10-01,Wang,A1,cited by applicant
996,11784043,132,20150259791,2015-09-01,Hausmann et al.,A1,cited by applicant
997,10848384,5,20090024426,2009-01-01,Chang,A1,cited by examiner
998,9767316,2,20030140027,2003-07-01,Huttel et al.,A1,cited by applicant


In [10]:
foreign_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_foreign_citation.tsv"

for chunk in pd.read_csv(foreign_file, sep='\t', usecols=["patent_id", "citation_application_id"], chunksize=chunksize, on_bad_lines='skip'):
    print(f"Processing rows {total_rows} to {total_rows + len(chunk)}")
    
    # Convert to string if needed
    chunk["patent_id"] = chunk["patent_id"].astype(str)
    chunk["citation_application_id"] = chunk["citation_application_id"].astype(str)

    filtered_df = chunk[chunk["patent_id"].isin(valid_patent_ids) & chunk["citation_application_id"].isin(valid_patent_ids)]
    
    pyedges = filtered_df.rename(columns={'patent_id': 'source', 'citation_application_id': 'target'}) \
            .astype(str) \
            .to_dict(orient='records')
    
    # add edges
    for edge in tqdm(pyedges, mininterval=1.0):
        graph.add_edge(edge["source"], edge["target"])


    total_rows += len(chunk)
    
    # Optional: Save processed chunk to a new file
    # chunk.to_csv("output_file.tsv", mode='a', sep='\t', index=False, header=(total_rows == 0))  # Append mode

print(f"Finished processing {total_rows} rows.")
del(chunk)
del filtered_df
del pyedges
# 22 min

Processing rows 144025878 to 154025878


100%|██████████| 44222/44222 [00:00<00:00, 271666.41it/s]


Processing rows 154025878 to 164025878


100%|██████████| 43944/43944 [00:00<00:00, 299506.97it/s]


Processing rows 164025878 to 174025878


...

Processing rows 174025878 to 184025878


100%|██████████| 43517/43517 [00:00<00:00, 331835.02it/s]


Processing rows 184025878 to 185802964


100%|██████████| 7672/7672 [00:00<00:00, 766524.54it/s]

Finished processing 185802964 rows.


In [10]:
grant_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_patent_citation.tsv"
# applic_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_application_citation.tsv"
# foreign_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_foreign_citation.tsv"
# output_file = "/home/edgarlanoue/citationsData/data/g_us_patent_citation_full.tsv"

# Load the TSV file into a DataFrame
# start_row = 20000000# #1000000
# end_row = 40000000

# Use skiprows to skip the first 20 million rows, and nrows to read the next 20 million
grantPat_df = pd.read_csv(grant_file, sep='\t', on_bad_lines='skip', nrows=10000000, usecols=["patent_id", "citation_patent_id"])
grantPat_df

,patent_id,citation_patent_id
0,10000000,5093563
1,10000000,5751830
2,10000001,7804268
3,10000001,9022767
4,10000001,9090016
...,...,...
9999995,10513056,D608406
9999996,10513056,7653279
9999997,10513056,D615784
9999998,10513056,7841666


#### Traitements données

In [50]:
grantPat_df["patent_id"] = grantPat_df["patent_id"].astype(str)
grantPat_df["citation_patent_id"] = grantPat_df["citation_patent_id"].astype(str)

In [52]:
filtered_df = grantPat_df[grantPat_df["patent_id"].isin(valid_patent_ids) & grantPat_df["citation_patent_id"].isin(valid_patent_ids)]

In [53]:
filtered_df.shape

(9472687, 2)

In [86]:
filtered_df[len(filtered_df)-1:len(filtered_df)]["patent_id"]

9999999    10513056
Name: patent_id, dtype: object

In [54]:
pyedges = filtered_df.rename(columns={'patent_id': 'source', 'citation_patent_id': 'target'}) \
            .astype(str) \
            .to_dict(orient='records')

In [56]:
# add edges
for edge in tqdm(pyedges, mininterval=1.0):
    graph.add_edge(edge["source"], edge["target"])

100%|██████████| 9472687/9472687 [00:16<00:00, 587082.67it/s]


In [58]:
import sys

graph
print(sys.getsizeof(graph))  # Returns memory size in bytes


56


### CD-index

In [12]:
print(graph.ecount())
print(graph.vcount())
print("_is_graph_sane: ", graph._is_graph_sane())

112921856
8123317
_is_graph_sane:  True


35257198

In [11]:
graph

In [11]:
graph.prepare_for_searching()

In [19]:
from dateutil.relativedelta import relativedelta
def get_t_delta(vertex, years):
      vertex_datetime = pd.to_datetime(graph.timestamp(patent), unit='s')
      future_datetime = vertex_datetime + relativedelta(years=years)
      difference = future_datetime - vertex_datetime
      print(difference)
      return int(difference.total_seconds())

In [24]:
vertex_datetime1 = pd.to_datetime(graph.timestamp(patent), unit='s')
print(vertex_datetime1)
vertex_datetime1 + relativedelta(years=5)

1993-11-23 00:00:00


Timestamp('1998-11-23 00:00:00')

In [126]:
patent = "5264790" #"7470642" #"5264790" # "5379515"  #"5264790"   #"5379515" #"5177974" #'10000022'
print("iindex: ", graph.iindex(patent, int(datetime.timedelta(days=1825).total_seconds())))
print("in_degree: ", graph.in_degree(patent))
print("out_degree: ", graph.out_degree(patent))
print("out_edges: ", graph.out_edges(patent))
print("in_edges: ", graph.in_edges(patent))

print("CD-index_5: ", graph.cdindex(patent, int(datetime.timedelta(days=1825).total_seconds())))#get_t_delta(patent, years=5))) # 1000000000)) #int(datetime.timedelta(days=1825).total_seconds()))) # 0.5
# print("CD-index_10: ", graph.cdindex(patent, get_t_delta(patent, years=10))) 
print("CD-index_10: ", graph.cdindex(patent, int(datetime.timedelta(days=365*10).total_seconds())))

iindex:  5.0
in_degree:  31
out_degree:  2
out_edges:  ['4525670', '4810965']
in_edges:  ['5537032', '5567058', '5570013', '5648614', '5825176', '5873658', '5898301', '5899573', '5967669', '6053637', '6564635', '6924638', '6928883', '7045999', '7228951', '7265537', '7275462', '7290351', '7367714', '7429133', '7467702', '7699732', '7758459', '7766140', '7811005', '7878714', '8123013', '8172056', '8201315', '8226301', '8950944']
CD-index_5:  0.5555555555555556
CD-index_10:  0.4074074074074074


In [80]:
import pandas as pd

# Path to the .csv.gz file
file_path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/nature_disruption_open_access/data/analytical/patentsview_analytical_df.csv.gz"

# Read the compressed CSV file
nature_df = pd.read_csv(file_path, compression='gzip')


print(f"DataFrame contains {nature_df.shape[0]} rows and {nature_df.shape[1]} columns.")
# Display the first few rows of the DataFrame
nature_df.head()

In [145]:
valid_patent_ids_set = set(map(str, valid_patent_ids)) 
valid_patent_ids==valid_patent_ids_set

True

In [162]:
valid_patent_ids_set = set(map(str, valid_patent_ids))  # Ensure all valid IDs are strings
all_in_valid = set(nature_df["patent_id"].astype(str)).issubset(valid_patent_ids_set)
print(all_in_valid)


True


In [147]:
set_nature = set(nature_df["patent_id"].astype(str))

In [163]:
valid_patent_ids == set_nature

True

In [161]:
set_nature in valid_patent_ids

False

In [160]:
print(len(set_nature))
print(len(valid_patent_ids))

3912353
3912353


In [148]:
set_nature==valid_patent_ids

False

In [77]:
sorted = nature_df.sort_values(by="cited_total", ascending=True)

In [78]:
sorted[500000:520000]

,patent_id,grant_year,field_nber_category_id,field_nber_subcategory_id,cited_total,cited_self,cited_self_risk,cited_age_mean,cited_age_sd,invs_crr_age_mean_t,invs_crr_npatent_ids_mean_t,cd_5,cd_10,cd_2017y,zscores_median,nsc_upatents_t,nsc_upatents_cited_total_t,nsc_upatents_cited_nentropy_t,nsc_upatents_inventors_total_t
3653475,7851676,2010,3.0,33.0,1,0,1,6.0,0.0,5.833333,1.833333,-1.000000,NaN,-1.000000,NaN,983.0,8356.0,0.863706,2473.0
3580782,7470642,2008,1.0,19.0,1,0,1,5.0,0.0,7.666667,12.333333,0.000000,0.052632,0.052632,NaN,8141.0,130162.0,0.983120,23618.0
3474444,7371698,2008,4.0,46.0,1,1,1,3.0,0.0,2.000000,7.000000,0.200000,0.181818,0.181818,NaN,10129.0,149435.0,0.962289,28035.0
741725,4596184,1986,6.0,69.0,1,0,1,6.0,0.0,9.000000,42.000000,0.500000,0.500000,0.333333,NaN,6752.0,19913.0,0.991474,11111.0
833638,4740647,1988,1.0,19.0,1,0,1,11.0,0.0,0.500000,0.500000,0.428571,0.222222,0.187500,NaN,7668.0,36533.0,0.987685,16707.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227905,6211233,2001,3.0,31.0,1,0,1,4.0,0.0,21.000000,22.000000,0.750000,0.058824,0.291667,NaN,11207.0,83727.0,0.971937,35836.0
1515559,5469101,1995,4.0,41.0,1,0,0,2.0,0.0,0.000000,0.000000,0.400000,0.400000,0.400000,NaN,3105.0,20350.0,0.987630,6021.0
1515542,5453703,1995,4.0,43.0,1,0,1,6.0,0.0,6.000000,4.000000,-0.052632,0.114286,0.203390,NaN,3190.0,22339.0,0.988395,6665.0
2228207,6541918,2003,4.0,42.0,1,0,1,3.0,0.0,8.000000,9.000000,0.157895,0.178295,0.169591,NaN,3726.0,33146.0,0.976252,8621.0


In [70]:
nature_df[nature_df["patent_id"]==10000022]

,patent_id,grant_year,field_nber_category_id,field_nber_subcategory_id,cited_total,cited_self,cited_self_risk,cited_age_mean,cited_age_sd,invs_crr_age_mean_t,invs_crr_npatent_ids_mean_t,cd_5,cd_10,cd_2017y,zscores_median,nsc_upatents_t,nsc_upatents_cited_total_t,nsc_upatents_cited_nentropy_t,nsc_upatents_inventors_total_t


0.006024
0.021978
0.020330
Problème de matching...


0.555556	0.366667	0.397436

In [108]:
365*5

1825

### CD-index Funk

In [25]:
# pip install git+https://github.com/russellfunk/cdindex.git

  Cloning https://github.com/russellfunk/cdindex.git to c:\users\edgar\appdata\local\temp\pip-req-build-b85vhg60
  Resolved https://github.com/russellfunk/cdindex.git to commit 4eece64c892fdb2f161b95c76ca0770ed5c97155
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cdindex: filename=cdindex-1.0.20-cp311-cp311-win_amd64.whl size=26349 sha256=6c4104f7cf924db9f75a0cad30aa4898f79b3284720d3d50bdb6cc39de916caa
  Stored in directory: C:\Users\edgar\AppData\Local\Temp\pip-ephem-wheel-cache-nl7w9ltx\wheels\bc\fa\4c\9a580e1925fe60b661077936d5886bce4c9ec3d09672749e92
Successfully built cdindex
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/russellfunk/cdindex.git 'C:\Users\edgar\AppData\Local\Temp\pip-req-build-b85vhg60'


In [129]:
import cdindex

In [7]:
graph_f = cdindex.Graph()

In [8]:
type(graph_f)

cdindex.cdindex.Graph

In [8]:
# add vertices
for vertex in tqdm(pyvertices, mininterval=1.0):
    graph_f.add_vertex(vertex["name"], int(datetime.datetime.timestamp(vertex["time"])))

100%|██████████| 3912353/3912353 [24:14<00:00, 2689.89it/s]


In [9]:
del pyvertices
valid_patent_ids = set(patent_df["patent_id"])
del(patent_df)

In [10]:
chunksize = 10_000_000  # Read 10 million rows at a time
total_rows = 0  # Counter to track progress
grant_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_patent_citation.tsv"

for chunk in pd.read_csv(grant_file, sep='\t', usecols=["patent_id", "citation_patent_id"], chunksize=chunksize, on_bad_lines='skip'):
    print(f"Processing rows {total_rows} to {total_rows + len(chunk)}")
    
    # Convert to string if needed
    chunk["patent_id"] = chunk["patent_id"].astype(str)
    chunk["citation_patent_id"] = chunk["citation_patent_id"].astype(str)

    filtered_df = chunk[chunk["patent_id"].isin(valid_patent_ids) & chunk["citation_patent_id"].isin(valid_patent_ids)]
    filtered_df=filtered_df.drop_duplicates()
    
    pyedges = filtered_df.rename(columns={'patent_id': 'source', 'citation_patent_id': 'target'}) \
            .astype(str) \
            .to_dict(orient='records')
    
    # add edges
    for edge in tqdm(pyedges, mininterval=1.0):
        graph_f.add_edge(edge["source"], edge["target"])


    total_rows += len(chunk)
    
    # Optional: Save processed chunk to a new file
    # chunk.to_csv("output_file.tsv", mode='a', sep='\t', index=False, header=(total_rows == 0))  # Append mode

print(f"Finished processing {total_rows} rows.")
del(chunk)
del filtered_df
del pyedges
# 22 min

Processing rows 0 to 10000000


0it [00:00, ?it/s]


Processing rows 10000000 to 20000000


0it [00:00, ?it/s]


Processing rows 20000000 to 30000000


...

Processing rows 130000000 to 140000000


0it [00:00, ?it/s]


Processing rows 140000000 to 144025878


0it [00:00, ?it/s]

Finished processing 144025878 rows.


In [15]:
from dateutil.relativedelta import relativedelta

In [27]:
def get_t_delta(vertex, years):
      vertex_datetime = cdindex.datetime_from_timestamp(graph_f.timestamp(vertex))
      print("     vertex date:", vertex_datetime)
      future_datetime = vertex_datetime + relativedelta(years=years)
      print("     future date:", future_datetime)
      difference = future_datetime - vertex_datetime
      print("     difference:", difference)
      return int(difference.total_seconds())

In [77]:
get_t_delta(vertex=patent, years=10)

     vertex date: 1993-11-23 05:00:00
     future date: 2003-11-23 05:00:00
     difference: 3652 days, 0:00:00


315532800

In [82]:
patent = "5264790" #"5264790"   #"5379515" #"5177974" #'10000022'
print("iindex: ", graph_f.iindex(patent, int(datetime.timedelta(days=1825).total_seconds())))
print("in_degree: ", graph_f.in_degree(patent))
print("out_degree: ", graph_f.out_degree(patent))
print("out_edges: ", graph_f.out_edges(patent))
print("in_edges: ", graph_f.in_edges(patent))

print("CD-index_5: ", graph_f.cdindex(patent, get_t_delta(vertex=patent, years=5)))#int(datetime.timedelta(days=1825).total_seconds()))) # 1000000000)) #int(datetime.timedelta(days=1825).total_seconds()))) # 0.5
print("CD-index_10: ", graph_f.cdindex(patent, 3155250000)) #int(datetime.timedelta(days=1825*2).total_seconds()))) 

iindex:  5.0
in_degree:  25
out_degree:  2
out_edges:  ['4525670', '4810965']
in_edges:  ['5537032', '5567058', '5570013', '5648614', '5825176', '5873658', '5898301', '5899573', '5967669', '6053637', '6564635', '6924638', '6928883', '7045999', '7228951', '7265537', '7275462', '7290351', '7367714', '7429133', '7467702', '7699732', '7758459', '7766140', '7811005']
     vertex date: 1993-11-23 05:00:00
     future date: 1998-11-23 05:00:00
     difference: 1826 days, 0:00:00
CD-index_5:  0.5555555555555556
CD-index_10:  0.4098360655737705


## CD index implémentation

In [40]:
pd.Timestamp(datetime.datetime(1992, 1, 1))

Timestamp('1992-01-01 00:00:00')

In [127]:
# dummy vertices for python module tests
pyvertices= [{"name": "0Z", "time": pd.Timestamp(datetime.datetime(1992, 1, 1))},
                 {"name": "1Z", "time": pd.Timestamp(datetime.datetime(1992, 1, 1))},
                 {"name": "2Z", "time": pd.Timestamp(datetime.datetime(1993, 1, 1))},
                 {"name": "3Z", "time": pd.Timestamp(datetime.datetime(1993, 1, 1))},
                 {"name": "4Z", "time": pd.Timestamp(datetime.datetime(1995, 1, 1))},
                 {"name": "5Z", "time": pd.Timestamp(datetime.datetime(1997, 1, 1))},
                 {"name": "6Z", "time": pd.Timestamp(datetime.datetime(1998, 1, 1))},
                 {"name": "7Z", "time": pd.Timestamp(datetime.datetime(1999, 1, 1))},
                 {"name": "8Z", "time": pd.Timestamp(datetime.datetime(1999, 1, 1))},
                 {"name": "9Z", "time": pd.Timestamp(datetime.datetime(1998, 1, 1))},
                 {"name": "10Z", "time": pd.Timestamp(datetime.datetime(1997, 1, 1))}]

pyedges = [{"source": "4Z", "target": "2Z"},
               {"source": "4Z", "target": "0Z"},
               {"source": "4Z", "target": "1Z"},
               {"source": "4Z", "target": "3Z"},
               {"source": "5Z", "target": "2Z"},
               {"source": "6Z", "target": "2Z"},
               {"source": "6Z", "target": "4Z"},
               {"source": "7Z", "target": "4Z"},
               {"source": "8Z", "target": "4Z"},
               {"source": "9Z", "target": "4Z"},
               {"source": "9Z", "target": "1Z"},
               {"source": "9Z", "target": "3Z"},
               {"source": "10Z", "target": "4Z"}]

In [97]:
pyvertices

[{'name': '0Z', 'time': Timestamp('1992-01-01 00:00:00')},
 {'name': '1Z', 'time': Timestamp('1992-01-01 00:00:00')},
 {'name': '2Z', 'time': Timestamp('1993-01-01 00:00:00')},
 {'name': '3Z', 'time': Timestamp('1993-01-01 00:00:00')},
 {'name': '4Z', 'time': Timestamp('1995-01-01 00:00:00')},
 {'name': '5Z', 'time': Timestamp('1997-01-01 00:00:00')},
 {'name': '6Z', 'time': Timestamp('1998-01-01 00:00:00')},
 {'name': '7Z', 'time': Timestamp('1999-01-01 00:00:00')},
 {'name': '8Z', 'time': Timestamp('1999-01-01 00:00:00')},
 {'name': '9Z', 'time': Timestamp('1998-01-01 00:00:00')},
 {'name': '10Z', 'time': Timestamp('1997-01-01 00:00:00')}]

In [ ]:
# pyedges

In [130]:
# create graph
graph2 = cdindex.Graph()

In [131]:
type(graph2)

cdindex.cdindex.Graph

In [132]:
# add vertices
for vertex in pyvertices:
    graph2.add_vertex(vertex["name"], int(datetime.datetime.timestamp(vertex["time"])))

In [133]:
# add edges
for edge in pyedges:
    graph2.add_edge(edge["source"], edge["target"])

In [135]:
graph2.cdindex("4Z", int(datetime.timedelta(days=1825).total_seconds()))

0.16666666666666666

In [136]:
graph2.mcdindex("4Z", int(datetime.timedelta(days=1825).total_seconds()))

0.8333333333333333

## Code from text

In [4]:
# import tarfile
# import os

# # Path to the tar.gz file
# tar_path = r"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/nature_disruption_open_access.tar.gz"
# tar_path = r"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/nature_disruption_open_access.tar.gz"

# # Directory where the files will be extracted
# extract_to = r"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data"

# # Ensure the output directory exists
# os.makedirs(extract_to, exist_ok=True)

# # Open and extract the tar.gz file
# with tarfile.open(tar_path, "r:gz") as tar:
#     tar.extractall(path=extract_to)

# print(f"Files have been extracted to {extract_to}")


In [14]:
sum(nature_df["cited_total"])

35257198

In [6]:
print(set(nature_df["grant_year"])) # 1976-2010

{1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010}


In [7]:
nature_df[nature_df["patent_id"]==3930271]

,patent_id,grant_year,field_nber_category_id,field_nber_subcategory_id,cited_total,cited_self,cited_self_risk,cited_age_mean,cited_age_sd,invs_crr_age_mean_t,invs_crr_npatent_ids_mean_t,cd_5,cd_10,cd_2017y,zscores_median,nsc_upatents_t,nsc_upatents_cited_total_t,nsc_upatents_cited_nentropy_t,nsc_upatents_inventors_total_t
49183,3930271,1976,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [43]:
# # Path to the .csv.gz file
# file_path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/nature_disruption_open_access/data/analytical/unified_cdindex_df.csv.gz"

# # Read the compressed CSV file
# cdInd_bm_df = pd.read_csv(file_path, compression='gzip')


# print(f"DataFrame contains {nature_df.shape[0]} rows and {nature_df.shape[1]} columns.")
# # Display the first few rows of the DataFrame
# cdInd_bm_df.head()

## Patents View 

In [2]:
# Path to the .csv.gz file
file_path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_patent.tsv"

# Load the TSV file into a DataFrame
pV_df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

# Display the first few rows of the DataFrame
print(pV_df.shape)
pV_df.head()

C:\Users\edgar\AppData\Local\Temp\ipykernel_21928\3931349859.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pV_df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')


(8980130, 8)


,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


In [3]:
pV_df["patent_date"] = pd.to_datetime(pV_df["patent_date"])

# Filter rows where the year is between 1976 and 2010
filtered_df = pV_df[(pV_df["patent_date"].dt.year >= 1976) & (pV_df["patent_date"].dt.year <= 2010) & (pV_df["patent_type"] == "utility")]
filtered_df.shape

(3912382, 8)

In [4]:
filtered_df2 = filtered_df[filtered_df["patent_id"].notna()]

# Step 2: Ensure "patent_id" is treated as string for numeric filtering
filtered_df2["patent_id"] = filtered_df2["patent_id"].astype(str)
print(filtered_df2.shape)
# Step 3: Keep rows where "patent_id" contains only numeric values (without NaN)
filtered_df2 = filtered_df2[filtered_df2["patent_id"].str.isnumeric()]

# Step 4: Optionally, convert "patent_id" back to numeric type
filtered_df2["patent_id"] = pd.to_numeric(filtered_df2["patent_id"])
# filtered_df2[filtered_df2["patent_id"]=="RE31700"]
print(filtered_df2.shape)

(3912382, 8)
(3912353, 8)


Garde les brevets entre 1976 et 2010, de type utility, dont le patent_id est uniquement numérique!

In [5]:
filtered_df = filtered_df2
filtered_df

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
2084459,3930271,utility,1976-01-06,Golf glove,A,4,0,pftaps19760106_wk01.zip
2084460,3930272,utility,1976-01-06,Crib leg lock,A,3,0,pftaps19760106_wk01.zip
2084461,3930273,utility,1976-01-06,Bed safety side rail arrangement,A,24,0,pftaps19760106_wk01.zip
2084462,3930274,utility,1976-01-06,Assembly for use in recreational activities,A,7,0,pftaps19760106_wk01.zip
2084463,3930275,utility,1976-01-06,Method of fabricating a slipper,A,9,0,pftaps19760106_wk01.zip
...,...,...,...,...,...,...,...,...
5996875,7861312,utility,2010-12-28,MP3 player with digital rights management,B2,16,0,ipg101228.xml
5996876,7861313,utility,2010-12-28,"Information managing apparatus and method, rec...",B2,4,0,ipg101228.xml
5996877,7861314,utility,2010-12-28,Interactive key control system and method of m...,B2,34,0,ipg101228.xml
5996878,7861315,utility,2010-12-28,Method for microfabricating a probe with integ...,B2,26,0,ipg101228.xml


## Matching nature et pV

In [13]:
nature_df[nature_df["patent_id"]==6173685]

,patent_id,grant_year,field_nber_category_id,field_nber_subcategory_id,cited_total,cited_self,cited_self_risk,cited_age_mean,cited_age_sd,invs_crr_age_mean_t,invs_crr_npatent_ids_mean_t,cd_5,cd_10,cd_2017y,zscores_median,nsc_upatents_t,nsc_upatents_cited_total_t,nsc_upatents_cited_nentropy_t,nsc_upatents_inventors_total_t
2301359,6173685,2001,5.0,53.0,255,16,1,12.309804,5.609699,25.0,50.0,-0.001032,-0.004318,-0.006148,85.872505,5200.0,45380.0,0.984951,11659.0


In [7]:
filtered_df2[filtered_df2["patent_id"]== 6173685]

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
4315816,6173685,utility,2001-01-16,Air-fuel module adapted for an internal combus...,A,30,0,pftaps20010116_wk03.zip


## Citations data

In [6]:
# Path to your exported TSV file
grant_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_patent_citation.tsv"
applic_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_application_citation.tsv"
foreign_file= "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_foreign_citation.tsv"
# output_file = "/home/edgarlanoue/citationsData/data/g_us_patent_citation_full.tsv"

# Load the TSV file into a DataFrame
# start_row = 20000000# #1000000
# end_row = 40000000

# Use skiprows to skip the first 20 million rows, and nrows to read the next 20 million
grantPat_df = pd.read_csv(grant_file, sep='\t', on_bad_lines='skip', usecols=["patent_id"]) #, skiprows=range(1, start_row+1), nrows=end_row-start_row,)

appPat_df = pd.read_csv(applic_file, sep='\t', on_bad_lines='skip', usecols=["patent_id"])

forPat_df = pd.read_csv(foreign_file, sep='\t', on_bad_lines='skip', usecols=["patent_id"])


# Display the first few rows of the DataFrame
# grantPat_df[14000:14010]
# 5 min

C:\Users\edgar\AppData\Local\Temp\ipykernel_21928\1879615952.py:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  grantPat_df = pd.read_csv(grant_file, sep='\t', on_bad_lines='skip', usecols=["patent_id"]) #, skiprows=range(1, start_row+1), nrows=end_row-start_row,)


In [7]:
print(grantPat_df.shape)
print(appPat_df.shape)
print(forPat_df.shape)


(144025878, 1)
(70557842, 1)
(41777086, 1)


Patents 3815111 sur 144025878

Applications: 572593 sur 68636755

Foreign: 2022085 sur 41777086

total: 6 409 789... répétitions!

In [8]:
print(len(set(grantPat_df["patent_id"]) & set(filtered_df2["patent_id"]))) # 3815111

grantPat_fil = set(grantPat_df["patent_id"]) & set(filtered_df2["patent_id"])

del grantPat_df

3815111


In [9]:
appPat_df = appPat_df[appPat_df["patent_id"].notna()]

# Step 2: Ensure "patent_id" is treated as string for numeric filtering
appPat_df["patent_id"] = appPat_df["patent_id"].astype(str)
appPat_df = appPat_df[appPat_df["patent_id"].str.isnumeric()]
appPat_df["patent_id"] = pd.to_numeric(appPat_df["patent_id"])

print(len(set(appPat_df["patent_id"]) & set(filtered_df2["patent_id"]))) # application all : 572593

appPat_fil = set(appPat_df["patent_id"]) & set(filtered_df2["patent_id"])

del appPat_df

572593


In [10]:
forPat_df = forPat_df[forPat_df["patent_id"].notna()]

# Step 2: Ensure "patent_id" is treated as string for numeric filtering
forPat_df["patent_id"] = forPat_df["patent_id"].astype(str)
forPat_df = forPat_df[forPat_df["patent_id"].str.isnumeric()]
forPat_df["patent_id"] = pd.to_numeric(forPat_df["patent_id"])

print(len(set(forPat_df["patent_id"]) & set(filtered_df2["patent_id"]))) # foreign all : 2022085

forPat_fil = set(forPat_df["patent_id"]) & set(filtered_df2["patent_id"])

del forPat_df

2022085


In [ ]:
print(len(appPat_fil | grantPat_fil) == len(grantPat_fil) or len(appPat_fil | grantPat_fil) == len(appPat_fil))
print(len(forPat_fil | grantPat_fil) == len(grantPat_fil) or len(forPat_fil | grantPat_fil) == len(forPat_fil))
print(len(appPat_fil | forPat_fil) == len(forPat_fil) or len(appPat_fil | forPat_fil) == len(appPat_fil))

False
False
False


: 

ils ajoutent tous quelque chose

In [42]:
combined_set = forPat_fil | appPat_fil | grantPat_fil

In [43]:
len(combined_set)

3878259

In [44]:
dif_set = (set(filtered_df["patent_id"]) - combined_set)
len(dif_set)
# je ne sais pas d'ou vient ce 34094

34094

In [45]:
dif_set

{6291461,
 5242895,
 6291472,
 5242897,
 3932180,
 5636117,
 6291484,
 5242921,
 4849708,
 4587567,
 4849712,
 5767218,
 5636147,
 5636164,
 6291525,
 5242952,
 6160457,
 5767247,
 5767252,
 5767256,
 5767258,
 5767262,
 5636191,
 7733342,
 5767265,
 5767270,
 4194424,
 7733376,
 7340161,
 5243016,
 4587657,
 5767304,
 4456589,
 7078030,
 4456591,
 5767316,
 4456597,
 4456598,
 6946969,
 7078044,
 5243041,
 4456612,
 6291621,
 5767336,
 5243049,
 4456618,
 5374124,
 5767341,
 4587699,
 4980916,
 5243059,
 5767349,
 5767350,
 5767351,
 5767352,
 5767354,
 5767355,
 5243068,
 6291642,
 5243071,
 3932364,
 6291663,
 6291665,
 6291666,
 6291670,
 6291673,
 6291680,
 4325602,
 6947042,
 4456679,
 6160617,
 4456684,
 4456687,
 5898493,
 7733504,
 6291713,
 4325635,
 3932420,
 3932426,
 3932427,
 3932435,
 3932441,
 6291739,
 3932445,
 7078175,
 7078178,
 3932451,
 7078183,
 3932458,
 3932459,
 7078188,
 7078190,
 3932472,
 4456767,
 3932480,
 4456769,
 5112129,
 5898563,
 3932489,
 3932490,


In [30]:
filtered_df[filtered_df["patent_id"]==6291461]

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
4433266,6291461,utility,2001-09-18,Stephacidin antitumor antibiotics,A,5,0,pftaps20010918_wk38.zip


In [36]:
3930271 in grantPat_fil
grantPat_fil

True

In [138]:
test = grantPat_df[grantPat_df["patent_id"]==6173685]
test.shape

(271, 7)

In [151]:
test["citation_date"] = pd.to_datetime(test["citation_date"])

C:\Users\edgar\AppData\Local\Temp\ipykernel_20340\3533886845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["citation_date"] = pd.to_datetime(test["citation_date"])


In [164]:
test[(test["citation_date"].dt.year>1975) & (test["citation_date"].dt.year<2011)]

,patent_id,citation_sequence,citation_patent_id,citation_date,record_name,wipo_kind,citation_category
16,6173685,59,3989066,1976-11-01,Sturman et al.,NaN,NaN
17,6173685,60,3995652,1976-12-01,Belart et al.,NaN,NaN
18,6173685,61,4009695,1977-03-01,Ule,NaN,NaN
19,6173685,62,4046112,1977-09-01,Deckard,NaN,NaN
20,6173685,63,4064855,1977-12-01,Johnson,NaN,NaN
...,...,...,...,...,...,...,...
266,6173685,309,5669355,1997-09-01,Gibson et al.,NaN,NaN
267,6173685,310,5673669,1997-10-01,Maley et al.,NaN,NaN
268,6173685,311,5697342,1997-12-01,Anderson et al.,NaN,NaN
269,6173685,312,5713315,1998-02-01,Jyoutaki et al.,NaN,NaN


In [ ]:
patent_df[patent_df["patent_id"]=="6173685"]

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
4315816,6173685,utility,2001-01-16 00:00:00+00:00,Air-fuel module adapted for an internal combus...,A,30,0,pftaps20010116_wk03.zip


In [37]:
from tqdm import tqdm
# Define data types for the selected columns
dtype_dict = {
    "patent_id": "int64",  # Assuming this column is always numeric
    "citation_patent_id": "string",  # Keep as string to preserve non-numeric data
    "citation_date": "string",  # Parse as datetime later
}

# Load only the required columns
columns_to_load = ["patent_id", "citation_patent_id", "citation_date"]

# Use chunks to handle large data
chunk_size = 10_000_000  # Adjust chunk size based on memory
chunks = pd.read_csv(output_file, usecols=columns_to_load, dtype=dtype_dict, chunksize=chunk_size, delimiter='\t')

# Process chunks
df_list = []
for chunk in tqdm(chunks):
    # Parse citation_date as datetime
    chunk["citation_date"] = pd.to_datetime(chunk["citation_date"], errors="coerce")
    
    # Append processed chunk to the list
    df_list.append(chunk)

# Combine all chunks into a single DataFrame (optional)
df = pd.concat(df_list, ignore_index=True)

# Display the result
print(df.info(memory_usage="deep"))

12it [02:40, 13.38s/it]


ValueError: invalid literal for int() with base 10: 'D1000000'